# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [12]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

url = 'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=0&sort=popularity'

headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [13]:
#We already Imported the pertinent libraries above. Furthermore, initial API endpoint URL has been established as well. 

#3. Now let's make a request to obtain data from the first page. 
response = requests.get(url,headers=headers)
results = response.json()

#Flattened data can be found here. 
flattened_data = json_normalize(results)
flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,...,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,tracking_information.metrigo_impression_urls,tracking_information.impression_beacon,tracking_information.source,outfits
0,HA115G00H-J11,SLIDE BRASIL - Sandales de bain - crocus rose,"[35/36, 37/38, 39/40, 41/42]",havaianas-slide-brasil-mules-ha115g00h-j11,[{'path': 'HA/11/5G/00/HJ/11/HA115G00H-J11@9.j...,Havaianas,False,"[{'sku': 'HA115G00H-J11', 'url_key': 'havaiana...","[{'key': 'discountRate', 'value': '-30%', 'tra...",shoe,...,"27,95 €","19,65 €",False,False,False,False,[https://ccp-et.adtechlab.zalan.do/event/sbv?z...,https://ccp-et.adtechlab.zalan.do/event/sbv?z=...,ccp,NaN
1,PUC22G0AM-C11,PREMIUM-KAROTTENJEANS MIT ZIERRISSEN 05684525 ...,"[36, 38, 40, 42, 44, 46]",pullandbear-premium-karottenjeans-mit-zierriss...,[{'path': 'PU/C2/2G/0A/MC/11/PUC22G0AM-C11@20....,PULL&BEAR,False,"[{'sku': 'PUC22G0AM-C11', 'url_key': 'pullandb...","[{'key': 'discountRate', 'value': '-40%', 'tra...",clothing,...,"29,99 €","17,99 €",False,False,False,False,[https://ccp-et.adtechlab.zalan.do/event/sbv?z...,https://ccp-et.adtechlab.zalan.do/event/sbv?z=...,ccp,NaN
2,LE222O055-C11,CREWNECK GRAPHIC 2 PACK - T-shirt imprimé - wh...,"[XS, S, M, L, XL, XXL, 3XL]",levisr-crewneck-graphic-2-pack-t-shirt-basique...,[{'path': 'LE/22/2O/05/5C/11/LE222O055-C11@9.1...,Levi's®,False,"[{'sku': 'LE222O055-C11', 'url_key': 'levisr-c...","[{'key': 'discountRate', 'value': '-15%', 'tra...",clothing,...,"34,95 €","29,65 €",False,False,False,False,[https://ccp-et.adtechlab.zalan.do/event/sbv?z...,https://ccp-et.adtechlab.zalan.do/event/sbv?z=...,ccp,NaN
3,NI112O08W-Q11,AIR MAX 200 - Baskets basses - black,"[38.5, 39, 40, 40.5, 41, 42, 42.5, 43, 44, 44....",nike-sportswear-air-max-200-baskets-basses-ni1...,[{'path': 'NI/11/2O/08/WQ/11/NI112O08W-Q11@8.j...,Nike Sportswear,False,"[{'sku': 'NI112O08W-Q11', 'url_key': 'nike-spo...","[{'key': 'discountRate', 'value': 'Jusqu’à -30...",shoe,...,"124,95 €","86,95 €",True,False,True,False,NaN,NaN,NaN,NaN
4,HU752D02V-Q11,GEXTER - Ceinture - black,"[85, 90, 95, 100, 105, 110, 115, 120]",hugo-gexter-ceinture-black-hu752d02v-q11,[{'path': 'HU/75/2D/02/VQ/11/HU752D02V-Q11@7.j...,HUGO,True,"[{'sku': 'HU752D02V-Q11', 'url_key': 'hugo-gex...","[{'key': 'discountRate', 'value': '-20%', 'tra...",accessoires,...,"69,95 €","55,95 €",False,False,False,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,TOB22S00X-C11,ORIGINAL ZIPTHRU - veste en sweat zippée - mot...,"[XS, M, XL]",tommy-jeans-original-zipthru-veste-en-sweat-to...,[{'path': 'TO/B2/2S/00/XC/11/TOB22S00X-C11@18....,Tommy Jeans,False,"[{'sku': 'TOB22S00X-C11', 'url_key': 'tommy-je...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,...,"89,95 €","71,95 €",False,False,False,False,NaN,NaN,NaN,NaN
80,PI912O021-A11,Baskets basses - white,"[40, 41, 42, 44, 45, 46, 47, 48]",pier-one-baskets-basses-white-pi912o021-a11,[{'path': 'PI/91/2O/02/1A/11/PI912O021-A11@2.j...,Pier One,False,"[{'sku': 'PI912O021-A11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-10%', 'tra...",shoe,...,"19,99 €","17,94 €",False,False,False,False,NaN,NaN,NaN,NaN
81,TOB22E00V-B11,SCANTON PANT - Chino - pumice stone,"[28x30, 28x32, 29x30, 29x32, 29x34, 30x30, 30x...",tommy-jeans-scanton-pant-chino-tob22e00v-b11,[{'path': 'TO/B2/2E/00/VB/11/TOB22E00V-B11@8.j...,Tommy Jeans,False,"[{'sku': 'TOB22E00V-B11', 'url_key': 'tommy-je...","[{'key': 'discountRate', 'value': '-30%', 'tra...",clothing,...,"89,95 €","62,95 €",False,False,False,False,NaN,NaN,NaN,NaN
82,LE222S038-Q11,GRAPHIC HOODIE - Sweat à capuche - black,"[S, M, L, XL, XXL]",levisr-graphic-hoodie-sweatshirt-black-le222s0...,[{'path': 'LE/22/2S/03/8Q/11/LE222S038-Q11@7.j...,Levi's®,False,"[{'sku': 'LE222

In [14]:
#4. Let's find out the total page count in the 1st page data.
total_pages = results['pagination']['page_count']
total_pages

800

In [15]:
#5. Now we will use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.
df = pd.DataFrame()

for i in range(total_pages):
    k = 84*i
    url = f'https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset={k}&sort=popularity'
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    response = requests.get(url, headers = headers)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    try:
        flattened_data1=flattened_data1.set_index('sku')
        df = df.append(flattened_data1, sort = True)
    except:
        pass

In [16]:
#6. Finally, let's print the data we just got. 
df

,amount,brand_name,delivery_promises,family_articles,flags,is_premium,media,name,outfits,price.base_price,...,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,tracking_information.impression_beacon,tracking_information.metrigo_impression_urls,tracking_information.source,url_key
sku,,,,,,,,,,,,,,,,,,,,,
HA115G00H-J11,NaN,Havaianas,[],[],"[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'HA/11/5G/00/HJ/11/HA115G00H-J11@9.j...,SLIDE BRASIL - Sandales de bain - crocus rose,NaN,NaN,...,False,False,"27,95 €","19,65 €",shoe,"[35/36, 37/38, 39/40, 41/42]",https://ccp-et.adtechlab.zalan.do/event/sbv?z=...,[https://ccp-et.adtechlab.zalan.do/event/sbv?z...,ccp,havaianas-slide-brasil-mules-ha115g00h-j11
PUC22G0AM-C11,NaN,PULL&BEAR,[],[],"[{'key': 'discountRate', 'value': '-40%', 'tra...",False,[{'path': 'PU/C2/2G/0A/MC/11/PUC22G0AM-C11@20....,PREMIUM-KAROTTENJEANS MIT ZIERRISSEN 05684525 ...,NaN,NaN,...,False,False,"29,99 €","17,99 €",clothing,"[36, 38, 40, 42, 44, 46]",https://ccp-et.adtechlab.zalan.do/event/sbv?z=...,[https://ccp-et.adtechlab.zalan.do/event/sbv?z...,ccp,pullandbear-premium-karottenjeans-mit-zierriss...
LE222O055-C11,NaN,Levi's®,[],[],"[{'key': 'discountRate', 'value': '-15%', 'tra...",False,[{'path': 'LE/22/2O/05/5C/11/LE222O055-C11@9.1...,CREWNECK GRAPHIC 2 PACK - T-shirt imprimé - wh...,NaN,NaN,...,False,False,"34,95 €","29,65 €",clothing,"[XS, S, M, L, XL, XXL, 3XL]",https://ccp-et.adtechlab.zalan.do/event/sbv?z=...,[https://ccp-et.adtechlab.zalan.do/event/sbv?z...,ccp,levisr-crewneck-graphic-2-pack-t-shirt-basique...
NI112O08W-Q11,NaN,Nike Sportswear,[],[],"[{'key': 'discountRate', 'value': 'Jusqu’à -30...",False,[{'path': 'NI/11/2O/08/WQ/11/NI112O08W-Q11@8.j...,AIR MAX 200 - Baskets basses - black,NaN,NaN,...,True,False,"124,95 €","86,95 €",shoe,"[38.5, 39, 40, 40.5, 41, 42, 42.5, 43, 44, 44....",NaN,NaN,NaN,nike-sportswear-air-max-200-baskets-basses-ni1...
HU752D02V-Q11,NaN,HUGO,[],[],"[{'key': 'discountRate', 'value': '-20%', 'tra...",True,[{'path': 'HU/75/2D/02/VQ/11/HU752D02V-Q11@7.j...,GEXTER - Ceinture - black,NaN,NaN,...,False,False,"69,95 €","55,95 €",accessoires,"[85, 90, 95, 100, 105, 110, 115, 120]",NaN,NaN,NaN,hugo-gexter-ceinture-black-hu752d02v-q11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OA344E07D-N11,NaN,Oakley,[],"[{'sku': 'OA344E07D-N11', 'url_key': 'oakley-m...","[{'key': 'campaign', 'value': 'Prix Mini', 'tr...",False,[{'path': 'OA/34/4E/07/DN/11/OA344E07D-N11@2.j...,MOD5 EUROPE - Casque - dark brush,NaN,NaN,...,False,False,"159,95 €","159,95 €",equipment,[],NaN,NaN,NaN,oakley-mod5-europe-casque-dark-brush-oa344e07d...
C6642B03R-912,NaN,Craft,[],"[{'sku': 'C6642B03R-912', 'url_key': 'craft-vi...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'C6/64/2B/03/R9/12/C6642B03R-912@1.1...,VISIBILITY VEST - Veste sans manches - yellow,NaN,NaN,...,False,False,"39,95 €","31,95 €",equipment,[S],NaN,NaN,NaN,craft-visibility-vest-veste-sans-manches-jaune...
PX844E00E-Q11,NaN,POC,[],"[{'sku': 'PX844E00E-Q11', 'url_key': 'poc-auri...","[{'key': 'discountRate', 'value': '-10%', 'tra...",False,[{'path': 'PX/84/4E/00/EQ/11/PX844E00E-Q11@13....,AURIC CUT - Casque - matt black,NaN,NaN,...,False,False,"169,95 €","152,95 €",equipment,[59-62],NaN,NaN,NaN,poc-auric-cut-casque-px844e00e-q11


## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [17]:
#1. Extracting the trending brand from our df.
df.brand_name.value_counts().index[0]

'Jack & Jones'

In [18]:
#2. Extracting the product with the highest discount. 

#Our data is still text. Let's convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

#Defining a Discount Ammount Column to later examine which product has the highest discount. 
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df1=df.copy()

#Group our SKUs by the ammount of money discounted from the original price.
total_disc=df1.groupby(['name']).sum().discount_amount

In [29]:
#Sort every SKU by ascending order and show the article with the most discount applied to it. 
total_disc.sort_values(ascending=False).index[0]

"Veste d'hiver - black"

In [39]:
#3. Getting the sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

#Let's first define a column named sum_discounted_prices.
sum_discounted_prices = df['price.promotional'].astype(float).sum()

#Then we'll define a column named sum_original_prices.
sum_original_prices = df['price.original'].astype(float).sum()

#Finally, let's make the ratio. 
ratio = sum_discounted_prices/sum_original_prices
print('The sum of discounts of all goods is: {:.2%}'.format(ratio))

The sum of discounts of all goods is: 81.70%
